# [실습 프로젝트]

- **다음과 같은 요구사항을 Gradio ChatInterface로 구현합니다**

- 주제: 맞춤형 여행 일정 계획 어시스턴트
- 기능: 
   - OpenAI Chat Completion API와 LangChain을 활용하여 사용자의 선호도에 맞는 여행 일정을 생성
   - LCEL을 사용하여 단계별 프롬프트 체인 구성 (사용자 입력 분석 -> 일정 생성 -> 세부 계획 수립)
   - 채팅 히스토리 사용하여 답변 생성
   - Gradio 인터페이스를 통해 사용자와 대화형으로 상호작용

- 주요 포인트:

   1. **모델 매개변수 최적화**
      - temperature=0.7: 적당한 창의성을 유지하면서 일관된 응답 생성
      - top_p=0.9: 높은 확률의 토큰만 선택하여 응답의 품질 향상
      - presence_penalty와 frequency_penalty: 반복적인 응답을 줄이고 다양한 제안 생성

   2. **시스템 프롬프트 설계**
      - 여행 플래너로서의 역할과 응답 가이드라인을 명확히 정의
      - 구체적인 정보를 포함하도록 지시
      - 한국어 응답 명시

   3. **메모리 관리**
      - Gradio 또는 LangChain 메모리 기능을 사용하여 대화 컨텍스트 유지
      - 이전 대화 내용을 바탕으로 연속성 있는 응답 생성

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# 필수 라이브러리 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import gradio as gr
import os

api_key = os.getenv("OPENAI_API_KEY")

summarize_template = [
    ("assistant", """당신은 여행 요청 분석 전문가입니다. 
    사용자의 메시지에서 여행과 관련된 핵심 정보를 추출해주세요."""),
    ("user", """다음 내용에서 여행 관련 핵심 정보를 추출해주세요:
    - 여행 기간/날짜
    - 인원수
    - 예산
    - 선호 지역/테마
    - 특별 요구사항
    
    텍스트: {user_input}""")
]
summarize_prompt = ChatPromptTemplate.from_messages(summarize_template)

schedule_template = [
    ("assistant", """당신은 여행 일정 구성 전문가입니다.
    추출된 정보를 바탕으로 기본 일정 프레임워크를 생성해주세요."""),
    ("user", """다음 정보를 바탕으로 기본 일정 구조를 만들어주세요:
    {user_input}
    
    포함사항:
    - 일자별 주요 활동
    - 이동 경로
    - 숙박 지역""")
]
schedule_prompt = ChatPromptTemplate.from_messages(schedule_template)

planning_template = [
            ("assistant", """당신은 맞춤형 여행 일정 계획 AI 어시스턴트입니다.
            다음 가이드라인을 따라 상세한 여행 계획을 수립해주세요:
            
            기본 원칙:
            • 한국어로 답변
            • 한국 내 여행지 추천
            • 실용적이고 실현 가능한 계획 수립
            • 최신 데이터 기반 답변
            • 숙소와 식당 등 15kg이상 반려견 동반이 가능한 곳
            
            포함사항:
            • 상세 일정표 (시간대별)
            • 예산 계획 및 비용 분석
            • 교통편 및 이동 방법
            • 숙박 추천 및 예약 팁
            • 날씨별 준비사항
            • 현지 맛집 및 명소 추천
            • 비상 연락처 및 안전 정보
            
            출력 형식:
            • 구조화된 일정표
            • 예산 요약표
            • 체크리스트"""),
            MessagesPlaceholder("chat_history"),
            ("user", "다음 기본 일정을 바탕으로 상세 여행 계획을 수립해주세요: {user_input}"),
        ]
planning_prompt = ChatPromptTemplate.from_messages(planning_template)

output_parser = StrOutputParser()
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.7, 
    top_p=0.9,
    presence_penalty=0.3,
    frequency_penalty=0.3,
)

summarize_chain = summarize_prompt | model | output_parser
schedule_chain = schedule_prompt | model | output_parser
planning_chain = planning_prompt | model | output_parser

def answer_invoke(message, history):

    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    # 요약
    summarized = summarize_chain.invoke({"user_input": message})
    # print(f"요약 결과: {summarized}")
    
    # 일정 생성
    schedule = schedule_chain.invoke({"user_input": summarized})
    # print(f"일정 생성 결과: {schedule}")

    # 세부 계획 수립
    plan_detail = planning_chain.invoke({
        "chat_history": history_messages,
        "user_input": schedule
    })

    return plan_detail

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    type="messages",
    title="맞춤형 여행 일정 계획 어시스턴트", # 채팅 인터페이스의 제목
    description="""
        해로와의 행복한 한국 내 여행 계획을 도와드리는 AI 어시스턴트입니다.
        
        **사용법:**
        • 여행 기간, 인원, 예산, 선호 지역을 알려주세요
        • 예시: "2박3일 제주도 여행, 커플, 예산 100만원, 자연경관 선호"
        
        **제공 서비스:**
        • 상세 일정표 • 예산 계획 • 교통편 안내 • 숙박 추천 • 맛집 정보
        """,
        examples=[
            "2박3일 부산 여행 계획해줘, 예산은 70만원이고 해산물을 좋아해",
            "1박2일 경주 역사문화 여행, 가족 4명, 예산 50만원",
            "3박4일 강원도 자연휴양 여행, 커플, 예산 120만원"
        ],
    )

# Gradio 인터페이스 실행
if __name__ == "__main__":
    demo.launch()

In [ ]:
demo.close()